### FinRAG Demo with Gradio, Weaviate and Ollama

#### This notebook expects the Ollama server to be backed by GPU with 16GB of memory.

#### This is a lite version that uses Weaviate's embedded instance.

In [ ]:
!pip install pip gradio weaviate_client wget ijson -Uq

In [ ]:
import gradio as gr
from huggingface_hub import InferenceClient
import weaviate.classes as wvc
import weaviate
from weaviate.auth import AuthApiKey
import logging
import os
import requests
import json
import ijson
import weaviate

ollama_api_endpoint = os.getenv("OLLAMA_HOST", "http://ollama.ollama")
ollama_vectorizer_model = model = "all-minilm"
ollama_generative_model="granite3-dense:8b"

logging.basicConfig(level=logging.ERROR)
logging.info(f'OLLAMA_API_ENDPOINT = {ollama_api_endpoint}')

In [ ]:
def connect_weaviate_embedded():
    logging.basicConfig(level=logging.ERROR)
    logging.info('Connecting to Weaviate embedded instance')
    client = weaviate.connect_to_embedded(
        environment_variables={"ENABLE_MODULES": "text2vec-ollama,generative-ollama"},
        version="1.25.6"
    )
    return client

In [ ]:
client = connect_weaviate_embedded()

if client.is_ready():
    logging.info('')
    logging.info(f'Found {len(client.cluster.nodes())} Weaviate nodes.')
    logging.info('')
    for node in client.cluster.nodes():
        logging.info(node)
        logging.info('')
    logging.info(f'client.get_meta(): {client.get_meta()}')
else:
    logging.error("Client is not ready")

In [ ]:
client.collections.delete_all()

In [ ]:
def download_data():
    try:
      os.stat("data/symbols.json")
      logging.info("Symbols already downloaded")
    except:
      logging.info("Downloading symbols...")
      url = "https://people.redhat.com/bkozdemb/downloads/symbols.json"
      wget.download(url, "data/symbols.json")

In [ ]:
def ingest_data(client):

    # ===== Define the collection =====
    symbols = client.collections.create(
        name="Symbols",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_ollama(
            api_endpoint=ollama_api_endpoint,
            model=ollama_vectorizer_model
        ),
        generative_config=wvc.config.Configure.Generative.ollama(
            api_endpoint=ollama_api_endpoint,
            model=ollama_generative_model
        )
    )

    # Settings for displaying the import progress
    counter = 0
    interval = 1000  # Print progress every this many records; should be bigger than the batch_size

    logging.info("JSON streaming, to avoid running out of memory on large files...")
    with client.batch.fixed_size(batch_size=50) as batch:
        with open("data/symbols.json", "rb") as f:
            objects = ijson.items(f, "item")
            for obj in objects:
                properties = {
                    "Symbol": obj["Symbol"],
                    "Name": obj["Name"],
                    "Description": obj["Description"],
                    "CIK": obj["CIK"],
                    "Exchange": obj["Exchange"],
                    "Currency": obj["Currency"],
                    "Country": obj["Country"],
                    "Sector": obj["Sector"], 
                    "Industry": obj["Industry"],
                    "Address": obj["Address"],
                    "FiscalYearEnd": obj["FiscalYearEnd"],
                    "LatestQuarter": obj["LatestQuarter"],
                    "MarketCapitalization": obj["MarketCapitalization"],
                    "BookValue": obj["BookValue"],
                    "EBITDA": obj["EBITDA"],
                    "PERatio": obj["PERatio"],
                    "PEGRatio": obj["PEGRatio"],
                    "DividendPerShare": obj["DividendPerShare"],
                    "DividendYield": obj["DividendYield"],
                    "EPS": obj["EPS"],
                    "RevenuePerShareTTM": obj["RevenuePerShareTTM"],
                    "ProfitMargin": obj["ProfitMargin"],
                    "OperatingMarginTTM": obj["OperatingMarginTTM"],
                    "ReturnOnAssetsTTM": obj["ReturnOnAssetsTTM"],
                    "ReturnOnEquityTTM": obj["ReturnOnEquityTTM"],
                    "RevenueTTM": obj["RevenueTTM"],
                    "GrossProfitTTM": obj["GrossProfitTTM"],
                    "DilutedEPSTTM": obj["DilutedEPSTTM"],
                    "QuarterlyEarningsGrowthYOY": obj["QuarterlyEarningsGrowthYOY"],
                    "QuarterlyRevenueGrowthYOY": obj["QuarterlyRevenueGrowthYOY"],
                    "AnalystTargetPrice": obj["AnalystTargetPrice"],
                    "AnalystRatingStrongBuy": obj["AnalystRatingStrongBuy"],
                    "AnalystRatingBuy": obj["AnalystRatingBuy"],
                    "AnalystRatingHold": obj["AnalystRatingHold"],
                    "AnalystRatingSell": obj["AnalystRatingSell"],
                    "AnalystRatingStrongSell": obj["AnalystRatingStrongSell"],
                    "TrailingPE": obj["TrailingPE"],
                    "ForwardPE": obj["ForwardPE"],
                    "PriceToSalesRatioTTM": obj["PriceToSalesRatioTTM"],
                    "PriceToBookRatio": obj["PriceToBookRatio"],
                    "EVToRevenue": obj["EVToRevenue"],
                    "EVToEBITDA": obj["EVToEBITDA"],
                    "Beta": obj["Beta"],
                    "fiftytwoWeekHigh": obj["52WeekHigh"],
                    "fiftytwoWeekLow": obj["52WeekLow"],
                    "fiftyDayMovingAverage": obj["50DayMovingAverage"],
                    "twohundredDayMovingAverage": obj["200DayMovingAverage"],
                    "SharesOutstanding": obj["SharesOutstanding"],
                    "DividendDate": obj["DividendDate"],
                    "ExDividendDate": obj["ExDividendDate"]
                }
                batch.add_object(
                    collection="Symbols",
                    properties=properties,
                    # If you Bring Your Own Vectors, add the `vector` parameter here
                    # vector=obj.vector["default"]
                )

                # Calculate and display progress
                counter += 1
                if counter % interval == 0:
                    logging.info(f"Imported {counter} of 7116 stock symbols.")


    logging.info(f"Finished importing {counter} symbols.")
    return symbols

In [ ]:
def semantic_search(query='computers', limit=2) -> dict:
    print(f'\nSemantic Search, query = {query}.')
    print(f'limit = {limit}')
    response = symbols.query.near_text(
        query=query,
        limit=limit
    )

    return_list = []
    for i in range(limit):
        return_list.append(response.objects[i].properties['name'])
    return return_list

In [ ]:
def generative_search(query='computers', task=None, limit=2) -> str:
    print(f'\nPerforming generative search, query = {query}, limit = {limit}.')
    print(f'Prompt: {task}')
    print(f'limit = {limit}')
    response = symbols.generate.near_text(
        query=query,
        limit=limit,
        grouped_task=task
    )
    return response.generated

In [ ]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.ERROR)
    download_data()
    symbols = ingest_data(client)
    #
    # Build the Gradio user interface.
    #
    with gr.Blocks(title='Summarizing Financial Data using RAG') as demo:
            gr.Markdown("""# Summarizing Financial Data using Retrieval Augmented Generation (RAG).""")
            semantic_examples = [
                ["Computers"],
                ["Computer Software"],
                ["Pharmaceuticals"],
                ["Consumer Products"],
                ["Commodities"],
                ["Retail"],
                ["Manufacturing"],
                ["Energy"],
                ["National Defense"],
                ["Auto Makers"]
            ]
            gr.Markdown("""### Begin with a search.""")
            semantic_input_text = gr.Textbox(label="Enter a search concept or choose an example below:", value=semantic_examples[0][0])
            gr.Examples(semantic_examples,
                fn=semantic_search,
                inputs=semantic_input_text, label="Example search concepts:"
                )
            limit_slider = gr.Slider(label="Adjust the query return limit. (Optional)",value=2, minimum=1, maximum=5, step=1)
            vdb_button = gr.Button(value="Search the financial vector database.")
            vdb_button.click(fn=semantic_search, inputs=[semantic_input_text, limit_slider], outputs=gr.Textbox(label="Search Results (Filters = Name)"))
            
            prompt_examples = [
                ["Generate a paragraph that summarizes the given information from a financial perspective for the fiscal year end of December 2024."],
                ["Summarize the information from a financial investment perspective."],
                ["Summarize the potential financial investment risks and rewards."]
            ]

            gr.Markdown("""### Summarize""")
            generative_search_prompt_text = gr.Textbox(label="Enter a summarization task or choose an example below.", value=prompt_examples[0][0])
            gr.Examples(prompt_examples,
                fn=generative_search,
                inputs=[generative_search_prompt_text]
            )
            button = gr.Button(value="Generate the summary.")
            button.click(fn=generative_search,
            inputs=[semantic_input_text, generative_search_prompt_text, limit_slider],
            outputs=gr.Textbox(label="Summary"))
            
    demo.queue(max_size=10)
    demo.launch(server_name='0.0.0.0', server_port=8081, share=True)

